TO DO

- is it possible to add captaincy bonus?
- add an optional constraint to allow only one defender from any given team (risk mitigation)?

In [1]:
import pandas as pd
import numpy as np
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable

In [2]:
# function to solve the optimization problem
def optimize(df, max_price, expected_column):
    
    # PRELIMINARIES
    
    # Create the model
    model = LpProblem(name="FPL", sense=LpMaximize)    
    variables = [LpVariable(name=f'{ix}', cat='Binary') for ix in df.index]
    prices = [df.loc[ix,'now_cost']/10.0 for ix in df.index]
    # measure of player quality
    expected_points = [df.loc[ix,expected_column] for ix in df.index]
    goalkeepers = [1.0 if df.loc[ix,'element_type']==1 else 0.0 for ix in df.index]
    defenders = [1.0 if df.loc[ix,'element_type']==2 else 0.0 for ix in df.index]
    midfielders = [1.0 if df.loc[ix,'element_type']==3 else 0.0 for ix in df.index]
    forwards = [1.0 if df.loc[ix,'element_type']==4 else 0.0 for ix in df.index]
    teams = []
    for team in df['team'].unique():
        teams.append( [1.0 if df.loc[ix,'team']==team else 0.0 for ix in df.index] )

    # CONSTRAINTS

    # select 11 players
    model += lpSum(variables) == 11
    # set maximum price for starting 11
    model += np.dot(prices,variables) <= max_price
    # only 1 goalkeeper
    model += np.dot(goalkeepers,variables) == 1
    # at least 3 defenders
    model += np.dot(defenders,variables) >= 3
    # at most 5 defenders
    model += np.dot(defenders,variables) <= 5
    # at most 5 midfielders
    model += np.dot(midfielders,variables) <= 5
    # at least 1 forward
    model += np.dot(forwards,variables) >= 1
    # at most 3 forwards
    model += np.dot(forwards,variables) <= 3
    # max 3 players from any given team
    for team in teams:
        model += np.dot(team,variables) <= 3
    
    # OBJECTIVE
    # if possible, SHOULD ADD CAPTAIN'S DOUBLE POINTS TO THE OBJECTIVE
    model += np.dot(expected_points,variables)

    # SOLVE OPTIMIZATION
    
    status = model.solve()
    print(f'Status: {LpStatus[model.status]}')
    #print(f'Mean total points per gameweek: {model.objective.value()}')

    players = [int(str(var)) for var in model.variables() if var.value()==1]
    dream_team = df.loc[players]
    cost = (dream_team['now_cost']/10.0).sum()
    exp_points = dream_team[expected_column].sum() + dream_team[expected_column].max()
    print(f'Cost: {cost}')
    if expected_column == 'adjusted points per game':
        print(f'Expected points per week: {exp_points}')
    elif expected_column == 'xPoints_next10':
        exp_points = exp_points / np.min((10, 38-gameweek))
        print(f'Expected points per week: {exp_points}')
    else:
        print(f'Expected points: {exp_points}')
    display(dream_team[['element_type','web_name','now_cost','games played',expected_column]].sort_values('element_type'))

In [3]:
gameweek = 34
# fetch FPL data
filepath = '../data/fpl/data_week' + str(gameweek) + '.csv'
df = pd.read_csv(filepath, index_col=0)
df

,adjusted points,adjusted points per game,assists,assists_week1,assists_week10,assists_week11,assists_week12,assists_week13,assists_week14,assists_week15,...,xPoints week 34,xPoints week 4,xPoints week 5,xPoints week 6,xPoints week 7,xPoints week 8,xPoints week 9,xPoints_next10,yellow_cards,gameweek 44 prediction
id,,,,,,,,,,,,,,,,,,,,,
1,10.789989,2.697497,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.753954,0,0.0
2,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.128080,0,0.0
3,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.942890,0,0.0
4,50.411000,3.600786,1,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,10.5,3.6,3.8,2.4,6.8,4.071,14.860736,3,0.0
5,42.259956,2.641247,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.941341,1.0,NaN,NaN,NaN,NaN,NaN,15.114708,3,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
661,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.330727,0,0.0
685,4.890833,2.445416,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.081689,1,0.0
686,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.391980,0,0.0


In [4]:
# only consider players that have played at least some minimum number of games
df = df[df['games played']>=5]
df

,adjusted points,adjusted points per game,assists,assists_week1,assists_week10,assists_week11,assists_week12,assists_week13,assists_week14,assists_week15,...,xPoints week 34,xPoints week 4,xPoints week 5,xPoints week 6,xPoints week 7,xPoints week 8,xPoints week 9,xPoints_next10,yellow_cards,gameweek 44 prediction
id,,,,,,,,,,,,,,,,,,,,,
4,50.411000,3.600786,1,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,10.500000,3.600000,3.800000,2.400000,6.800000,4.071000,14.860736,3,0.0
5,42.259956,2.641247,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.941341,1.000000,NaN,NaN,NaN,NaN,NaN,15.114708,3,0.0
6,87.628000,3.370308,8,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,1.000000,3.600000,2.000000,18.553866,0,0.0
7,53.728235,2.442192,2,0.0,NaN,NaN,NaN,NaN,NaN,0.0,...,2.135335,NaN,NaN,2.332871,NaN,NaN,NaN,14.878304,8,0.0
12,16.336369,1.815152,2,NaN,NaN,0.0,0.0,0.0,1.0,NaN,...,2.406335,NaN,NaN,NaN,NaN,NaN,NaN,10.998386,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461,68.279852,2.731194,1,0.0,0.0,0.0,NaN,0.0,0.0,0.0,...,1.000000,4.418731,2.382085,NaN,5.048812,NaN,NaN,12.063986,0,0.0
470,59.101494,3.283416,4,NaN,1.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,3.861196,12.757897,4,0.0
475,127.969739,3.877871,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.331484,5.274923,0.328340,11.375777,4.195247,3.331484,3.961196,21.599094,3,0.0


In [5]:
optimize(df, 86, f'gameweek {gameweek+1} prediction')

Status: Optimal
Cost: 85.9
Expected points: 70.65013122558594


,element_type,web_name,now_cost,games played,gameweek 35 prediction
id,,,,,
475,1,Sá,53,33.0,4.719661
135,2,Chilwell,56,6.0,4.812013
237,2,Alexander-Arnold,84,29.0,8.016706
360,2,Doherty,46,15.0,4.323334
44,2,Cash,52,32.0,4.282429
527,2,Chalobah,47,18.0,4.322432
233,3,Salah,133,31.0,6.928017
240,3,Jota,83,30.0,5.456180
359,3,Son,110,30.0,7.770307


In [6]:
optimize(df, 86, 'adjusted points per game')

Status: Optimal
Cost: 85.30000000000001
Expected points per week: 68.699167038689


,element_type,web_name,now_cost,games played,adjusted points per game
id,,,,,
156,1,Butland,44,6.0,4.454500
122,2,Alonso,56,24.0,4.915750
135,2,Chilwell,56,6.0,6.175292
237,2,Alexander-Arnold,84,29.0,6.271308
256,2,Cancelo,71,31.0,5.732337
233,3,Salah,133,31.0,7.609821
240,3,Jota,83,30.0,5.328407
265,3,Foden,78,24.0,5.168259
359,3,Son,110,30.0,6.001804


In [7]:
optimize(df, 86, f'xPoints_next10')

Status: Optimal
Cost: 85.6
Expected points per week: 84.23474967479706


,element_type,web_name,now_cost,games played,xPoints_next10
id,,,,,
199,1,Meslier,48,33.0,22.504966
122,2,Alonso,56,24.0,22.957652
135,2,Chilwell,56,6.0,25.376520
224,2,Matip,51,27.0,21.501765
237,2,Alexander-Arnold,84,29.0,36.974303
123,3,Jorginho,57,26.0,22.666233
212,3,Maddison,70,31.0,24.289580
215,3,Barnes,66,27.0,23.932966
233,3,Salah,133,31.0,32.157582


In [8]:
df = df.drop([135])
optimize(df, 86, f'xPoints_next10')

Status: Optimal
Cost: 86.0
Expected points per week: 83.56757205724716


,element_type,web_name,now_cost,games played,xPoints_next10
id,,,,,
199,1,Meslier,48,33.0,22.504966
122,2,Alonso,56,24.0,22.957652
224,2,Matip,51,27.0,21.501765
237,2,Alexander-Arnold,84,29.0,36.974303
527,2,Chalobah,47,18.0,21.761326
212,3,Maddison,70,31.0,24.289580
215,3,Barnes,66,27.0,23.932966
233,3,Salah,133,31.0,32.157582
359,3,Son,110,30.0,36.517263


In [9]:
df = df.drop([527])
optimize(df, 86, f'xPoints_next10')

Status: Optimal
Cost: 85.6
Expected points per week: 83.48390710353851


,element_type,web_name,now_cost,games played,xPoints_next10
id,,,,,
199,1,Meslier,48,33.0,22.504966
119,2,Azpilicueta,56,22.0,22.373149
122,2,Alonso,56,24.0,22.957652
224,2,Matip,51,27.0,21.501765
237,2,Alexander-Arnold,84,29.0,36.974303
123,3,Jorginho,57,26.0,22.666233
212,3,Maddison,70,31.0,24.289580
215,3,Barnes,66,27.0,23.932966
233,3,Salah,133,31.0,32.157582


In [ ]:
df = df.drop([205])
optimize(df, 86, f'xPoints_next10')

In [ ]:
df = df.drop([360])
optimize(df, 90, f'gameweek {gameweek+1} prediction')